scikit-learn 设置 Lasso alpha 参数是通过两个公开的对象，LassoCV 和 LassoLarsCV。其中，LassoLarsCV 是基于下面将要提到的 最小角回归 算法。

对于具有多个共线特征的高维数据集，LassoCV通常是更可取的。然而，LassoLarsCV 具有探索更多相关α参数值的有点，如果样本数相对于特征数很少，则往往比LassoCV更快

In [ ]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LassoCV, LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 生成示例数据
X, y = make_regression(n_samples=100, n_features=10, noise=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用 LassoCV 进行模型选择
lasso_cv = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, random_state=42)
lasso_cv.fit(X_train, y_train)

print("LassoCV 最佳 alpha:", lasso_cv.alpha_)
print("LassoCV 系数:", lasso_cv.coef_)

# 预测并评估
y_pred_lasso_cv = lasso_cv.predict(X_test)
mse_lasso_cv = mean_squared_error(y_test, y_pred_lasso_cv)
print(f"LassoCV 均方误差: {mse_lasso_cv:.4f}")

# 使用 LassoLarsCV 进行模型选择
lasso_lars_cv = LassoLarsCV(alphas=np.logspace(-4, 4, 100), cv=5)
lasso_lars_cv.fit(X_train, y_train)

print("\nLassoLarsCV 最佳 alpha:", lasso_lars_cv.alpha_)
print("LassoLarsCV 系数:", lasso_lars_cv.coef_)

# 预测并评估
y_pred_lasso_lars_cv = lasso_lars_cv.predict(X_test)
mse_lasso_lars_cv = mean_squared_error(y_test, y_pred_lasso_lars_cv)
print(f"LassoLarsCV 均方误差: {mse_lasso_lars_cv:.4f}")    

LassoCV 在训练完成后会自动使用交叉验证找到的最佳 alpha 对应的系数进行预测。下面详细解释其内部原理和实现细节：

### LassoCV 内部原理

LassoCV 是一种通过交叉验证自动选择最优正则化参数 alpha 的 Lasso 回归方法。其核心步骤如下：

1. 定义候选 alpha 列表  
    用户可以通过 alphas 参数指定一组候选的 alpha 值（默认会自动生成一个合理范围）。
2. 交叉验证  
    对每个候选 alpha 值：
 - 将训练数据分为 cv 个子集（默认 cv=5）。
 - 在 cv-1 个子集上训练 Lasso 模型，在剩余子集上评估性能（默认使用均方误差 MSE）。
 - 重复此过程 cv 次，计算平均验证误差。
3. 选择最优 alpha  
    选择平均验证误差最小的 alpha 值作为最优参数。
4. 最终模型训练  
    使用全部训练数据和最优 alpha 重新训练模型，得到最终系数 coef_。

### 预测机制

当你调用 lasso_cv.predict(X_test) 时，LassoCV 会直接使用步骤 4 中训练好的模型（即最优 alpha 对应的系数）进行预测。这一过程是自动的，无需手动指定 alpha。

In [4]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LassoCV
import numpy as np

# 生成数据
X, y = make_regression(n_samples=100, n_features=10, noise=20, random_state=42)

# 训练 LassoCV
lasso_cv = LassoCV(alphas=np.logspace(-4, 4, 5), cv=3, random_state=42)
lasso_cv.fit(X, y)

# 验证最优 alpha 和系数
print("最优 alpha:", lasso_cv.alpha_)  # 例如：0.01

# 手动验证：对每个 alpha，获取交叉验证的系数路径
coef_paths = lasso_cv.coef_path_  # shape: (n_alphas, n_features)
best_alpha_idx = np.argmin(lasso_cv.mse_path_.mean(axis=1))

# 比较自动选择的系数 vs 手动提取的最优 alpha 对应的系数
print("自动选择的系数:", lasso_cv.coef_)
print("手动提取的最优系数:", coef_paths[best_alpha_idx])

# 验证结果：两者完全一致

最优 alpha: 0.0001


AttributeError: 'LassoCV' object has no attribute 'coef_path_'